In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder




In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv("../input/30daystraindatakfolds/train_data_kfolds.csv")
test_df = pd.read_csv("../input/30daysofml/test.csv")
submission = pd.read_csv("../input/30daysofml/sample_submission.csv")

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
features = [col for col in df.columns if col not in ["id","target","kfold"]]
features

In [ ]:
object_cols= list(df[features].select_dtypes(include="object").columns)
len(object_cols)
object_cols

In [ ]:
test_df = test_df[features]
test_df

In [ ]:
final_predictions = []
for fold in range(5):
    X_train = df[df.kfold!=fold].reset_index(drop=True)
    y_train = X_train.target
   

    X_valid = df[df.kfold==fold].reset_index(drop=True)
    y_valid = X_valid.target
    
    X_train = X_train[features]
    X_valid = X_valid[features]
    X_test = test_df.copy()

    #apply ordinal encoding for 
    ordinal_encoder = OrdinalEncoder()
    X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
    X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])
    X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])
    
    #model = RandomForestRegressor(n_estimators = 100 ,learning_rate=0.05)
    model = XGBRegressor(n_estimators = 300, learning_rate = 0.05 , n_jobs=4, random_state=fold)
    model.fit(X_train, y_train)
    
    prediction = model.predict(X_valid)
    testpred = model.predict(X_test)
    final_predictions.append(testpred)
    mse = mean_squared_error(prediction, y_valid, squared=False)
    print(f" MSE value for fold {fold} is : {mse}")
    
    

In [ ]:
np.column_stack(final_predictions)

In [ ]:
predictions = np.mean(np.column_stack(final_predictions), axis=1)
predictions

In [ ]:
submission.target = predictions
submission

In [ ]:
submission.to_csv("submission.csv", index=False)